# Graph loading

Different methods of loading (batched) graph tensors for keras model in- and output.

In [1]:
import tensorflow as tf
from kgcnn.literature.GIN import make_model

In [2]:
inputs = [
    {"shape": (None,), "name": "node_number", "dtype": "float32", "ragged": True},
    {"shape": (None, 2), "name": "edge_indices", "dtype": "int64", "ragged": True}
]
outputs = {"shape": [], "name": "graph_labels", "dtype": "float32", "ragged": False}

In [3]:
model = make_model(
    inputs=inputs,
    output_mlp={"units": 1, "activation": "linear"}
)
model.compile(loss="mean_absolute_error")

INFO:kgcnn.utils.models:Updated model kwargs:
INFO:kgcnn.utils.models:{'name': 'GIN', 'inputs': [{'shape': (None,), 'name': 'node_number', 'dtype': 'float32', 'ragged': True}, {'shape': (None, 2), 'name': 'edge_indices', 'dtype': 'int64', 'ragged': True}], 'input_embedding': {'node': {'input_dim': 95, 'output_dim': 64}}, 'gin_mlp': {'units': [64, 64], 'use_bias': True, 'activation': ['relu', 'linear'], 'use_normalization': True, 'normalization_technique': 'batch'}, 'gin_args': {}, 'depth': 3, 'dropout': 0.0, 'verbose': 10, 'last_mlp': {'use_bias': [True, True, True], 'units': [64, 64, 64], 'activation': ['relu', 'relu', 'linear']}, 'output_embedding': 'graph', 'output_to_tensor': True, 'output_mlp': {'use_bias': True, 'units': 1, 'activation': 'linear'}}


### 1. As TensorFlow tensor

Via `MemoryGraphList` method `tensor()`:

In [4]:
from kgcnn.data.datasets.ESOLDataset import ESOLDataset
dataset = ESOLDataset()
dataset.clean(["node_number", "edge_indices"])

INFO:kgcnn.data.download:Checking and possibly downloading dataset with name ESOL
INFO:kgcnn.data.download:Dataset directory located at C:\Users\patri\.kgcnn\datasets
INFO:kgcnn.data.download:Dataset directory found. Done.
INFO:kgcnn.data.download:Dataset found. Done.
INFO:kgcnn.data.ESOL:Found SDF C:\Users\patri\.kgcnn\datasets\ESOL\delaney-processed.sdf of pre-computed structures.
INFO:kgcnn.data.ESOL: ... read molecules 0 from 1128
INFO:kgcnn.data.ESOL: ... read molecules 1000 from 1128
INFO:kgcnn.data.ESOL:Property edge_indices is an empty list for graph 934.


array([934])

In [5]:
tensor_input = dataset.tensor(inputs)
print([x.shape for x in tensor_input])

[TensorShape([1127, None]), TensorShape([1127, None, 2])]


In [6]:
tensor_output = tf.expand_dims(dataset.tensor(outputs), axis=-1)
print(tensor_output.shape)

(1127, 1)


In [7]:
model.fit(tensor_input, tensor_output, epochs=5)

Epoch 1/5


C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/gin_2/gather_nodes_outgoing_2/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/model/gin_2/gather_nodes_outgoing_2/Reshape:0", shape=(None, 64), dtype=float32), dense_shape=Tensor("gradient_tape/model/gin_2/gather_nodes_outgoing_2/Cast:0", shape=(2,), dtype=int32))) to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(
C:\Users\patri\anaconda3\envs\gcnn_keras_test\lib\site-packages\tensorflow\python\framework\indexed_slices.py:444: UserWarning: Converting sparse IndexedSlices(IndexedSlices(indices=Tensor("gradient_tape/model/gin_1/gather_nodes_outgoing_1/Reshape_1:0", shape=(None,), dtype=int64), values=Tensor("gradient_tape/model/gin_1/gather_nodes_outgoing_1/Reshape:0", shape=(None, 64), dtype=float32), dense_shap

36/36 [==============================] - 6s 38ms/step - loss: 1.5457
Epoch 2/5
36/36 [==============================] - 1s 36ms/step - loss: 1.2378
Epoch 3/5
36/36 [==============================] - 1s 34ms/step - loss: 1.1142
Epoch 4/5
36/36 [==============================] - 1s 34ms/step - loss: 1.0818
Epoch 5/5
36/36 [==============================] - 1s 36ms/step - loss: 1.0021


### 2. Via `keras.utils.Sequence`

For example `GraphBatchLoader` that inherits from `ks.utils.Sequence` and takes an iterable data object of type `list[dict]`.

In [8]:
from kgcnn.io.loader import GraphBatchLoader

In [9]:
loader = GraphBatchLoader(data=dataset, inputs=inputs, outputs=outputs)

In [11]:
model.fit(loader, epochs=5)

Epoch 1/5
36/36 [==============================] - 1s 38ms/step - loss: 0.9854
Epoch 2/5
36/36 [==============================] - 1s 39ms/step - loss: 0.9118
Epoch 3/5
36/36 [==============================] - 1s 38ms/step - loss: 0.8588
Epoch 4/5
36/36 [==============================] - 1s 38ms/step - loss: 0.8635
Epoch 5/5
36/36 [==============================] - 1s 41ms/step - loss: 0.8495


### 3. Via `tf.data `